In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import *

# Underlying set representation
S = TypeVar('S')

class Space(Generic[S]):

    # If names are not provided, the set is assumed global
    names: list[str]
    approx: str = 'exact'

    @property
    def ndim(self) -> int:
        return len(self.names)
    
    @property
    def vars(self):
        return self._vars(self.names)

    def __call__(self, s: S) -> S:
        return self.complement(self.empty())

class SetBuilder(Generic[S]):

    def __init__(self, f: Callable[[S], S]):
        self.f = f

    def __call__(self, s: S = None) -> S:
        s = self.f(s)
        return s
    
    def __mul__(self, rhs: SetBuilder[S]) -> SetBuilder[S]:
        """Set intersection"""
        return SetBuilder(lambda s: self.instersect(self(s), rhs(s)))

    def __add__(self, rhs: SetBuilder[S]) -> SetBuilder[S]:
        """Set union"""
        return SetBuilder(lambda s: self.union(self(s), rhs(s)))
    
    def __sub__(self, rhs: SetBuilder[S]) -> SetBuilder[S]:
        """Set difference"""
        return SetBuilder(lambda s: self.minus(self(s), rhs(s)))
    
    def __or__(self, rhs: SetBuilder[S] | tuple) -> SetBuilder[S]:
        if isinstance(rhs, SetBuilder[S]):
            return self.such_that(rhs)
        elif isinstance(rhs, tuple):
            return (self * rhs[0]) | rhs[1:] if rhs else self
        else:
            return NotImplemented

    @abstractmethod
    @staticmethod
    def empty(self, s: S) -> S: pass

    @abstractmethod
    @staticmethod
    def complement(self, s: S) -> S: pass

    @abstractmethod
    @staticmethod
    def instersect(self, s: S) -> S: pass

    @staticmethod
    def union(self, s1: S, s2: S) -> S:
        return self.complement(self.intersect(self.complement(s1), self.complement(s2)))
    
    @staticmethod
    def minus(self, s1: S, s2: S) -> S:
        return self.intersect(s1, self.complement(s2))

    @dataclass
    class _Var:
        idx: int
        name: str

        def __lt__(self, rhs: float) -> SetBuilder[S]:
            return (NotImplemented if not isinstance(rhs, (int, float)) else
                    SetBuilder(lambda s: s.lower_half(self.idx, rhs)))
        
        def __gt__(self, rhs: float) -> SetBuilder[S]:
            return (NotImplemented if not isinstance(rhs, (int, float)) else
                    lambda s: s.upper_half(self.idx, rhs))
    
    def _vars(self, names) -> list[_Var]:
        return [self._Var(i, name) for i, name in enumerate(names)] 

### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### 

class LevelSet(SetBuilder):
        
        def empty(self):
            return np
        
        def upper_half(self, axis: int, value: float) -> Set:
            return NotImplemented
    
        def lower_half(self, axis: int, value: float) -> Set:
            return NotImplemented

s = Set()
(((s | s.X < 0.5) | s.Y > 0.5) | s.Z < 0.5)

SyntaxError: invalid syntax (1358183413.py, line 88)

In [ ]:
S = Space(*'x y a v'.split())
X, Y, A, V = S.vars

s = S.subspace
s = s | X > 3, Y < 2

# s: lower 2 (upper 3 (ident s))

In [ ]:

class Solver(ABC):

    @abstractmethod
    def complement(self, a: Set) -> Set: pass

    @abstractmethod
    def intersect(self, a: Set) -> Set: pass
    
    @abstractmethod
    def reach(self, a: Set, b: Set) -> Set: pass
    
    @abstractmethod
    def rci(self, a: Set) -> Set: pass
